In [1]:
import pandas as pd
import shutil
import os
from datetime import datetime, timedelta
import sqlite3

In [2]:
df = pd.read_excel('Acionamentos.xlsx')
df2 = pd.read_excel('Ocorrencias.xlsx')
df3 = pd.read_excel('Operadores.xlsx')

In [ ]:
df.columns

In [4]:
df_clean = df[['Cobrador', 'Data Inclusão', 'Acionamento', 'ID', 'Fone/Destino']]
df_clean = pd.merge(df_clean, df2[['Acionamento', 'Classificação']], on='Acionamento', how='left')
df_clean = pd.merge(df_clean, df3[['Cobrador', 'Supervisor', 'Carteira']], on='Cobrador', how='left')
df_clean['Data Inclusão'] = pd.to_datetime(df_clean['Data Inclusão'])
df_clean['Data'] = df_clean['Data Inclusão'].dt.date

In [5]:
df_envio = df_clean[df_clean['Classificação'].isin(['CPC - TARGET', 'CPC', 'ALÔ', 'ACORDO', 'SEM CONTATO'])]

In [ ]:
data_inicio = datetime(2025, 1, 16) 
data_fim = datetime.today()  
df_envio['Data'] = pd.to_datetime(df_envio['Data']) 
df_envio = df_envio[(df_envio['Data'] >= data_inicio) & (df_envio['Data'] <= data_fim)]
df_envio.set_index('Cobrador', inplace=True)

In [ ]:
df_envio.to_excel('caminho/Registros DIA/Hoje.xlsx')

In [8]:
data_hoje = datetime.now().strftime('%Y-%m-%d')

In [9]:
novo_nome = f'{data_hoje}.xlsx'

In [ ]:
os.rename('caminho/Registros DIA/Hoje.xlsx', novo_nome)
shutil.move(novo_nome, 'caminho/Registros DIA')

In [11]:
lista_emails = ["exemplo@gmail.com"]

In [ ]:
import win32com.client as win32

# Criar uma instância do Outlook
outlook = win32.Dispatch('outlook.application')

# Criar um novo e-mail
email = outlook.CreateItem(0)

email.To = "; ".join(lista_emails) 

# Definir o destinatário, assunto e corpo do e-mail
email.Subject = "TARGETS ENTREGUES"
email.Body = "Segue o targets entregues a operação na data de hoje. Este email foi enviado de forma automática."

# Anexar o arquivo renomeado
caminho_arquivo = os.path.abspath(f'caminho/{novo_nome}')

email.Attachments.Add(caminho_arquivo)
# Enviar o e-mail
email.Send()

df_envio
print("Email enviado com sucesso!")